In [1]:
# test PPO, SAC for n=1
# repeat for other n new file/same file

In [1]:
import physics_environments
import numpy as np
from physics_environments import CartPendulumPhysicsConstants, CartPendulumPhysicsEnvironmentParams, CartPendulumPhysicsEnvironment, \
                                 RLCartPendulumEnvironment, RLCartPendulumEnvironmentParams

from jumanji import register, registered_environments
from jumanji.wrappers import VmapWrapper, AutoResetWrapper, VmapAutoResetWrapper
import jumanji

import jax
import jax.numpy as jnp
import time
import os

import lovely_jax as lj
lj.monkey_patch()

In [2]:
#jax.config.update('jax_debug_nans', False)
#jax.config.update('jax_debug_infs', False)
#jax.config.update("jax_enable_x64", False)
#jax.config.update('jax_disable_jit', True)

In [2]:
# Possible Parameter Inspirations: https://github.com/vwxyzjn/cleanrl/blob/master/cleanrl/ppo_continuous_action_isaacgym/ppo_continuous_action_isaacgym.py

%run /home/hbeyer/RL_Pendulum_Project/Stoix/stoix/systems/ppo/anakin/ff_ppo_continuous.py  \
\
 env=jumanji/cart_2_pendulum\
 env.solved_return_threshold=750.0\
\
 hydra.searchpath=[/home/hbeyer/RL_Pendulum_Project/Stoix/stoix/configs] \
 hydra.verbose=False\
\
 arch.seed=42\
 arch.total_num_envs=4096\
\
 system.rollout_length=16\
 system.critic_lr=0.003\
 system.actor_lr=0.003\
 system.vf_coef=0.5\
 system.ent_coef=0.001\
 system.epochs=4\
 system.num_minibatches=4\
\
 network.actor_network.pre_torso.activation=tanh\
 network.critic_network.pre_torso.activation=tanh\
\
 arch.total_timesteps=30_000_000\
 arch.num_eval_episodes=128\
 arch.num_evaluation=100\
\
 logger.use_wandb=False\
 logger.checkpointing.save_model=True\
 logger.kwargs.detailed_logging=True

 


Using Anakin System!
Changing the number of updates to 457: If you want to train for a specific number of updates, please set total_timesteps to None!


{
│   'logger': {
│   │   'base_exp_path': 'results',
│   │   'use_console': True,
│   │   'use_tb': False,
│   │   'use_json': False,
│   │   'use_neptune': False,
│   │   'use_wandb': False,
│   │   'kwargs': {
│   │   │   'project': None,
│   │   │   'tags': ['stoix'],
│   │   │   'detailed_logging': True,
│   │   │   'json_path': None,
│   │   │   'upload_json_data': False
│   │   },
│   │   'checkpointing': {
│   │   │   'save_model': True,
│   │   │   'save_args': {
│   │   │   │   'save_interval_steps': 1,
│   │   │   │   'max_to_keep': 1,
│   │   │   │   'keep_period': None,
│   │   │   │   'checkpoint_uid': None
│   │   │   },
│   │   │   'load_model': False,
│   │   │   'load_args': {'checkpoint_uid': ''}
│   │   }
│   },
│   'arch': {
│   │   'architecture_name': 'anakin',
│   │   'seed': 42,
│   │   'update_batch_size': 1,
│   │   'total_num_envs': 4096,
│   │   'total_timesteps': 30000000,
│   │   'num_updates': 457,
│   │   'evaluation_greedy': False,
│   │   'num_eval_episodes': 128,
│   │   'num_evaluation': 100,
│   │   'absolute_metric': True,
│   │   'num_envs': 4096,
│   │   'num_updates_per_eval': 4,
│   │   'devices': [CudaDevice(id=0)]
│   },
│   'system': {
│   │   'system_name': 'ff_ppo',
│   │   'actor_lr': 0.003,
│   │   'critic_lr': 0.003,
│   │   'rollout_length': 16,
│   │   'epochs': 4,
│   │   'num_minibatches': 4,
│   │   'gamma': 0.99,
│   │   'gae_lambda': 0.95,
│   │   'clip_eps': 0.2,
│   │   'ent_coef': 0.001,
│   │   'vf_coef': 0.5,
│   │   'max_grad_norm': 0.5,
│   │   'decay_learning_rates': False,
│   │   'standardize_advantages': True,
│   │   'kl_penalty_coef': 3.0,
│   │   'action_dim': 1,
│   │   'action_minimum': -25.0,
│   │   'action_maximum': 25.0
│   },
│   'network': {
│   │   'actor_network': {
│   │   │   'pre_torso': {
│   │   │   │   '_target_': 'stoix.networks.torso.MLPTorso',
│   │   │   │   'layer_sizes': [256, 256, 256, 256],
│   │   │   │   'use_layer_norm': False,
│   │   │   │   'activation': 'tanh'
│   │   │   },
│   │   │   'action_head': {'_target_': 'stoix.networks.heads.NormalAffineTanhDistributionHead'}
│   │   },
│   │   'critic_network': {
│   │   │   'pre_torso': {
│   │   │   │   '_target_': 'stoix.networks.torso.MLPTorso',
│   │   │   │   'layer_sizes': [256, 256, 256, 256],
│   │   │   │   'use_layer_norm': False,
│   │   │   │   'activation': 'tanh'
│   │   │   },
│   │   │   'critic_head': {'_target_': 'stoix.networks.heads.ScalarCriticHead'}
│   │   }
│   },
│   'env': {
│   │   'env_name': 'jumanji',
│   │   'observation_attribute': 'agent_inputs_global',
│   │   'multi_agent': False,
│   │   'scenario': {'name': 'cart_n_pendulum-v1', 'task_name': 'cart_2_pendulum'},
│   │   'kwargs': {
│   │   │   'params': {
│   │   │   │   'physics_env_params': {
│   │   │   │   │   'constants': {
│   │   │   │   │   │   'n': 2,
│   │   │   │   │   │   'g': 9.81,
│   │   │   │   │   │   'l': [0.3, 0.3],
│   │   │   │   │   │   'r': [0.2, 0.2],
│   │   │   │   │   │   'm': [0.8, 0.8],
│   │   │   │   │   │   'I': [0.011, 0.011],
│   │   │   │   │   │   'mu': [0.015, 0.015]
│   │   │   │   │   },
│   │   │   │   │   'env_type': 'compound pendulum',
│   │   │   │   │   'save_after_init': False,
│   │   │   │   │   'save_filename': 'env2n_compound pendulum_obj.pkl'
│   │   │   │   },
│   │   │   │   'training_generator_type': 'DefaultGenerator',
│   │   │   │   'dt': 0.01,
│   │   │   │   't_episode': 10.0,
│   │   │   │   'max_cart_acceleration': 25.0,
│   │   │   │   'cart_width': 0.25,
│   │   │   │   'track_width': 1.25,
│   │   │   │   'reward_exponent': 1.0,
│   │   │   │   '_init_phyiscs_env_from_file': False,
│   │   │   │   '_physics_env_init_filename': 'pendulum_obj_filename.pkl',
│   │   │   │   '_initialize_env_math': False
│   │   │   }
│   │   },
│   │   'eval_metric': 'episode_return',
│   │   'solved_return_threshold': 750.0
│   },
│   'num_devices': 1
}

orbax-checkpoint version: 0.10.1
[thread=MainThread] Failed to get flag value for EXPERIMENTAL_ORBAX_USE_DISTRIBUTED_PROCESS_ID.
[process=0][thread=MainThread] CheckpointManager init: checkpointers=<orbax.checkpoint._src.checkpointers.pytree_checkpointer.PyTreeCheckpointer object at 0x7f72de349900>, item_names=None, item_handlers=None, handler_registry=None
Configured `CheckpointManager` using deprecated legacy API. Please follow the instructions at https://orbax.readthedocs.io/en/latest/api_refactor.html to migrate.
Deferred registration for item: "default". Adding handler `<orbax.checkpoint._src.handlers.pytree_checkpoint_handler.PyTreeCheckpointHandler object at 0x7f72de34ac20>` for item "default" and save args `<class 'orbax.checkpoint._src.handlers.pytree_checkpoint_handler.PyTreeSaveArgs'>` and restore args `<class 'orbax.checkpoint._src.handlers.pytree_checkpoint_handler.PyTreeRestoreArgs'>` to `_handler_registry`.
Deferred registration for item: "metrics". Adding handler `<orba

PPO experiment completed
